In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from cartpole_environment import CartPoleEnv #'cartpole_environment' is how I named the gym file, possibly needs to be changed
import keras.backend as kb

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
###SETUP###
gamma = 0.95
num_actions = 2
exploration = 'egreedy'
experience_replay = True
decay_epsilon = True #This is to perform dynamic epsilon-greedy exploration by decaying the value of epsilon 
#with time

epsilon = 0.25  # Epsilon greedy parameter
epsilon_min = 0.01
decay_rate = 0.995
batch_size = 20  # Size of batch taken from replay buffer
max_steps_per_episode = 400

game = CartPoleEnv()

C:\Users\nikma\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
###BUILD THE ARCHITECTURE OF THE MODEL###
inputs = keras.Input(shape=(4,))
x = layers.Dense(24, activation = 'relu')(inputs)
x = layers.Dense(24, activation = 'relu')(x)
#x = layers.Dense(200, activation = 'relu')(x) #Let's see what happens when removing a layer
outputs = layers.Dense(2, activation = 'linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer = optimizer, loss = 'mse')

In [11]:
###Experience replay buffers###
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Maximum replay length
max_memory_length = 1000000
# Train the model after a fixed number of actions
update_after_actions = 10
run = 0

while True:  # Run until solved
    state = np.array(game.reset())
    episode_reward = 0
    run += 1
    n_steps = 0
    
    for timestep in range(1, max_steps_per_episode):
        #game.render() #Adding this line would show the attempts of the agent in a pop up window.
        frame_count += 1
        n_steps +=1
        
        if exploration == 'egreedy':
            if epsilon > np.random.rand(1)[0]:
                action = np.random.choice(num_actions)
            else:
                # Predict action Q-values from environment state
                action_probs = model.predict(np.array([state,]))
                # Take best action
                action = tf.argmax(action_probs[0]).numpy()

                # Decay probability of taking random action
            if decay_epsilon:
                epsilon *= decay_rate
                epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = game.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fixed number of frames and once batch size is reached
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = np.array([rewards_history[i] for i in indices])
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Q value = reward + discount factor * expected future reward
            y_train = model.predict(state_sample)
            for i in range(len(y_train)-2):
                if not done_sample[i]:
                    y_train[i][action_sample[i]] = rewards_sample[i] + gamma*np.max(model.predict(np.array([state_sample[i],])))
                else:
                    y_train[i][action_sample[i]] = 1 # If final frame set the value to 1
            
            #Train the model
            model.fit(state_sample, y_train, verbose = 0)
            
        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]
            
        # If don print the score of current run
        if done:
            print("Run:" + str(run) + ", Steps:" + str(n_steps))
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 150:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

#In case we need to end the game
#game.close()

Run:1, Steps:10
Run:2, Steps:8
Run:3, Steps:8
Run:4, Steps:10
Run:5, Steps:9
Run:6, Steps:10
Run:7, Steps:8
Run:8, Steps:9
Run:9, Steps:10
Run:10, Steps:9
Run:11, Steps:9
Run:12, Steps:8
Run:13, Steps:9
Run:14, Steps:10
Run:15, Steps:9
Run:16, Steps:9
Run:17, Steps:10
Run:18, Steps:10
Run:19, Steps:9
Run:20, Steps:9
Run:21, Steps:8
Run:22, Steps:10
Run:23, Steps:10
Run:24, Steps:10
Run:25, Steps:8
Run:26, Steps:9
Run:27, Steps:10
Run:28, Steps:10
Run:29, Steps:10
Run:30, Steps:10
Run:31, Steps:8
Run:32, Steps:9
Run:33, Steps:10
Run:34, Steps:10
Run:35, Steps:10
Run:36, Steps:8
Run:37, Steps:10
Run:38, Steps:9
Run:39, Steps:10
Run:40, Steps:10
Run:41, Steps:10
Run:42, Steps:10
Run:43, Steps:10
Run:44, Steps:9
Run:45, Steps:9
Run:46, Steps:9
Run:47, Steps:10
Run:48, Steps:10
Run:49, Steps:10
Run:50, Steps:10
Run:51, Steps:10
Run:52, Steps:9
Run:53, Steps:9
Run:54, Steps:10
Run:55, Steps:9
Run:56, Steps:10
Run:57, Steps:9
Run:58, Steps:9
Run:59, Steps:10
Run:60, Steps:9
Run:61, Steps:9
Ru

KeyboardInterrupt: 